In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

data = load_breast_cancer()
X = data.data
y = data.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

y = np.where(y == 0, -1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def polynomial_kernel(x1, x2, degree=3, coef0=1):
    return (np.dot(x1, x2.T) + coef0) ** degree

class SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y[idx]))
                    self.b -= self.lr * y[idx]

    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)

def polynomial_features(X, degree=3):
    poly = []
    for i in range(X.shape[0]):
        poly.append(np.power(X[i], degree))
    return np.array(poly)

X_train_poly = polynomial_features(X_train, degree=2)
X_test_poly = polynomial_features(X_test, degree=2)

svm = SVM(learning_rate=0.001, lambda_param=0.01, n_iters=2000)
svm.fit(X_train_poly, y_train)

y_pred = svm.predict(X_test_poly)

y_pred_binary = np.where(y_pred == 1, 1, 0)
y_true_binary = np.where(y_test == 1, 1, 0)

cm = confusion_matrix(y_true_binary, y_pred_binary)
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", classification_report(y_true_binary, y_pred_binary))

fpr, tpr, _ = roc_curve(y_true_binary, y_pred_binary)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, label='SVM (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve — SVM Polynomial Kernel')
plt.legend(loc='lower right')
plt.show()

Good afternoon, ma’am. This program builds a breast-cancer classifier using a Support Vector Machine (SVM) that I train myself with gradient updates, and I also evaluate it with a confusion matrix, a classification report, and an ROC curve. First, I load the built-in Breast Cancer Wisconsin dataset using load_breast_cancer(), which gives me $\mathbf{X}$ (the feature measurements from cell nuclei) and $\mathbf{y}$ (the labels: malignant or benign). I standardize all features with StandardScaler() so each column has mean 0 and variance 1; this helps SVM training because all features are on a comparable scale. I convert the labels from $\{0, 1\}$ to $\{-1, +1\}$ using np.where, since my SVM implementation expects $-1$ for one class and $+1$ for the other. Then I split the data into training and testing sets with an $80/20$ ratio using train_test_split and a fixed random state for reproducible results. I define a polynomial_kernel function for completeness, but in this script I actually expand the features explicitly instead of using the kernel trick. For that, I write a polynomial_features(X, degree=3) helper; here I call it with degree=2, which simply squares each feature (note: this version adds only power terms like $x_j^2$ and not cross terms $x_i x_j$). I transform both $\mathbf{X}_{\text{train}}$ and $\mathbf{X}_{\text{test}}$ into these quadratic features to let a linear SVM separate the data in this richer space. Next, I implement an SVM class with a learning rate, regularization parameter $\lambda_{\text{param}}$, and number of iterations. In fit, I initialize the weight vector $\mathbf{w}$ and bias $b$ to zero, then loop for the chosen number of iterations; for each training example, I check the hinge margin condition $y_i (\mathbf{w} \cdot \mathbf{x}_i - b) \ge 1$. If the point is correctly classified with margin, I only apply weight decay $\mathbf{w} \leftarrow \mathbf{w} - \text{lr} \cdot 2\lambda\mathbf{w}$; otherwise, I also add a corrective step that pushes the decision boundary toward the true side: $\mathbf{w} \leftarrow \mathbf{w} - \text{lr} \cdot (2\lambda\mathbf{w} - y_i \mathbf{x}_i)$ and $b \leftarrow b - \text{lr} \cdot y_i$. After training, predict computes the decision value $\mathbf{w} \cdot \mathbf{x} - b$ and returns its sign: $-1$ for malignant and $+1$ for benign (after our mapping). I train this SVM on the quadratic features for $2000$ iterations, then predict on the test set. For evaluation, I convert predictions and truths back to $\{0, 1\}$ to match scikit-learn’s metrics, compute the confusion matrix to see correct vs. incorrect counts, and print a classification report with precision, recall, and F1 per class. Finally, I plot an ROC curve: I get the false-positive rate and true-positive rate from roc_curve using the predicted labels, compute the AUC, and plot the ROC along with the diagonal baseline. This end-to-end pipeline shows: load and scale medical features, expand to polynomial (degree-2) features, train a linear SVM in that feature space via hinge-loss updates, evaluate with standard classification metrics, and visualize discrimination ability with an ROC curve.